Python Imlementation of:

***SCHILLINGER INSTRUMENT DESTRIBUTION***

for more information: Joseph Schillinger (1946): Schillinger System of Musical Composition.(New York: C. Fischer, Inc.)

Konstantin Svechtarov 2017©

In [1]:
# imports modules
%load_ext autoreload
%autoreload 2
import schillinger.rhythm as SR
import schillinger.pitch as SP
import itertools
import random
from music21 import *
%load_ext music21.ipython21 

In [19]:
#convert to notes
def convertToNotes(num_parts, linear_seq, start_pauses, time_meter, notes, duration_devider):
    
    #TS = str(time_meter[0])+'/'+str(time_meter[1])
    TS = str(time_meter[0])+'/'+str(4) # make it on qurters!
    tones_parts = [[]for x in range(num_parts)]
    
    #add pauses
    for i,sp in enumerate(start_pauses):
        if sp != -1:
            r = note.Rest()
            r.duration.quarterLength = sp*duration_devider
            tones_parts[i].append(r)
    
    #add notes
    for i, seq in enumerate(linear_seq):
        tone = notes[i%len(notes)]
        nota = pitch.Pitch(tone)
        #nota.octave = seq[0]+2
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = seq[1] * duration_devider
        tones_parts[seq[0]].append(nota)
    
    #add to stream
    s = stream.Score(id='mainScore')
    for seq, sequences in enumerate(tones_parts):
        p0 = stream.Part(id='part'+str(seq))
        p0.timeSignature = meter.TimeSignature(TS)
        p0.append(sequences)
        s.insert(0, p0)
    
    
    return s

In [20]:
# harmonize_bass func 
def harmonize_dt_bass(scales, diatonic_cycles_string, coeficients):
    SPG = SP.PitchGroup()
    bass = []
    harmonies = []
    for s in scales:
        temp_scale = [str(p) for p in s.pitches[:-1]]
        scale_expansion = SPG.expansions(temp_scale)
        #print(scale_expansion)
        C7 = scale_expansion[0]
        Cminus7 = scale_expansion[5]
        C3 = scale_expansion[4]
        Cminus3 = scale_expansion[1]
        C5 = scale_expansion[2]
        Cminus5 = scale_expansion[3]
        diatonic_cycles = []
        for i in diatonic_cycles_string:
            if i == "C3":
                diatonic_cycles.append(C3)
            elif i == "C5":
                diatonic_cycles.append(C5)
            elif i == "C7":
                diatonic_cycles.append(C7)
            elif i == "C-3":
                diatonic_cycles.append(Cminus3)
            elif i == "C-5":
                diatonic_cycles.append(Cminus5)
            elif i == "C-7":
                diatonic_cycles.append(Cminus7)
            else:
                print("diatonic_cycles_string Value Error!")
                
        diatonic_cycle = SP.pitch_cicles(temp_scale,diatonic_cycles, coeficients)
        harmonies.append(temp_scale)
        bass.append(diatonic_cycle)
    return bass, harmonies

In [42]:
#HARMONY

hm = scale.HarmonicMinorScale("C4")
do = scale.DorianScale("C4")
ms = scale.MixolydianScale("C4")
ls = scale.LydianScale("C4")

#hb = harmonize_dt_bass([hm], ["C3","C5","C7"], [2,1,1,2])
bass, scales = harmonize_dt_bass([hm], ["C3","C7","C5"], [2,1,1,2])
voice_amount = 3
theme_expansion_amount = 0
scale_expansion_amount = 1 # 1st order expansion for harmonizing is according to schillinger default (in thirds)

whole_seq = SP.get_whole_sequence_bass(bass, scales, voice_amount, theme_expansion_amount, scale_expansion_amount)
#print(whole_seq)

merged_seq = list(itertools.chain(*whole_seq)) # convert to [note1[harmony1], ... noteN[harmonyN]]
#print(list(itertools.chain(*merged_seq)))
flattened_notes = list(itertools.chain(*merged_seq))

In [46]:
#RHYTHM #ARRANGEMENT
sr_1 = SR.Type_I([7,5])
durations_pattern = [1,3,1] 
attack_pattern = [1,2,2,1,3,1,1,1] 
order_of_parts = [1,0]
num_parts = len(order_of_parts)


cts = SR.CTS(num_parts,order_of_parts,durations_pattern, attack_pattern)
#note_seq = flattened_notes
note_seq = ['F','C','D','F','A-']

In [49]:
s = convertToNotes(num_parts, cts.linear_sequence, cts.start_pauses, cts.metric, note_seq, 0.25)

In [50]:
s.show('musicxml')

In [8]:
#TEST1 cardcoded patterns

durations_pattern = [1,3,1] # rhythm (sr_1.resultant)
attack_pattern = [1,2,2,1,3,1,1,1] #instrument destribution
order_of_parts = [1,0]
num_parts = len(order_of_parts)
cts = SR.CTS(num_parts,order_of_parts,durations_pattern,attack_pattern)
note_seq = flattened_notes


In [9]:
#TEST2 schillinger rhythm patterns

sr_1 = SR.Type_I([3,2])
sr_2 = SR.Type_I([3,4])
group = SR.Grouping()
g = group.grouping_by_pairs(sr_1.resultant,sr_2.resultant,4,3)
print(group.balance)
random_attack_combination = SR.get_random_attack_combination(sr_1.resultant)
random_attack_combination_2 = SR.get_random_attack_combination(sr_2.resultant)

fraction = 2/4
dp = SR.slice_by_fraction(sr_1.resultant, fraction) # slice the resultant for fun

durations_pattern = group.balance # rhythm
attack_pattern = group.balance #instrument destribution

order_of_parts = [1,0,2]
num_parts = len(order_of_parts)
cts = SR.CTS(num_parts,order_of_parts,durations_pattern,attack_pattern)


[3, 1, 2, 2, 1, 3, 2, 1, 1, 2, 4]
